### Autograd 사용
- 어떤 tensor가 학습에 필요한 tensor라면 `backpropagation`을 통해 gradient를 구해야 함

- `tensor`의 gradient를 구하기 위한 조건

    - `requires_grad = True` 로 설정되어 있어야함, default는 False
    
    - `backpropagation`을 시작하는 지점의 output은 `scalar` 형태   
    
- `tensor`의 gradient를 계산하기 위해서는 output 지점의 tensor에서 `.backward()` 함수를 호출하면 됨

- gradient 값을 확인하려면 `requires_grad=True`로 생성한 tensor에 `.grad` 를 통해 확인할 수 있음